<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2022-10-26 16:55:17--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  28.6MB/s    in 52s     

2022-10-26 16:56:10 (26.8 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
wv_embeddings['dog'] 

array([ 0.6851772 , -1.2778991 , -0.41913974,  1.3623164 , -3.1675398 ,
        0.09950767,  0.6402681 , -1.1245339 , -0.6699619 , -0.6998852 ,
        0.4936771 , -0.40500194, -3.0706816 , -2.2809966 ,  0.85798043,
        2.7093108 ,  0.3492745 , -0.03494101, -0.22330493,  1.2290467 ,
        1.7755157 , -3.158358  , -0.6515983 ,  0.7224096 ,  2.3193083 ,
       -1.7969862 ,  0.40903398, -2.744604  , -1.7179952 , -0.914309  ,
       -0.75887376, -0.35140672, -0.5182776 , -1.9097351 , -0.8300773 ,
        0.02147918,  1.1783471 ,  0.03169126, -0.3069023 ,  1.6666299 ,
        0.6711357 , -2.1706133 , -0.11800487,  0.22336982, -1.2075394 ,
       -0.86297905, -0.63865614,  1.1733794 ,  0.10022762,  0.7017279 ,
        2.7290728 , -0.4640484 , -2.1719306 , -0.3562852 , -1.8449957 ,
        0.10270727,  1.1125596 , -0.8364318 ,  1.9513408 , -0.97937447,
        1.2650859 ,  0.06809282,  0.6477318 , -0.52431005, -0.6103959 ,
       -2.979829  , -0.7889965 , -0.11004248,  1.7603841 , -1.05

In [5]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место? 

In [6]:
# method most_simmilar
wv_embeddings.most_similar('dog')

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804807662964),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398118972778),
 ('bird', 0.7315489053726196),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349938392639),
 ('labrador', 0.7209131717681885)]

ОТВЕТ 1: Видим, что слово cat не входит, а входит слово cats.

In [7]:
wv_embeddings.most_similar('cat')

[('-vet', 0.7583354711532593),
 ('dog', 0.6852341294288635),
 ('/proc/version', 0.681588888168335),
 ('feline', 0.6779966354370117),
 ('nyan', 0.6685197353363037),
 ('com/schemas/2007/categories', 0.6678527593612671),
 ('-et', 0.6623897552490234),
 ('pajamas', 0.6485012173652649),
 ('tabby', 0.6465495824813843),
 ('catcowcat', 0.64402836561203)]

Хотя слово dog входит в топ-5 близких к cat.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [8]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [9]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """

    k = [embeddings[word] for word in tokenizer.tokenize(question) if word in embeddings]
        
    if k:
      return sum(k)/len(tokenizer.tokenize(question))
    else: 
      return np.zeros(shape=(dim, ))
    

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [10]:
question = "I love neural networks"
question_to_vec(question, embeddings = wv_embeddings, tokenizer = MyTokenizer())[2].round(2)

-0.96

ОТВЕТ 2: -0.96. Это третья(с индексом 2) компонента вектора предложения "I love neural networks".

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

ОТВЕТ: 1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [11]:
k = 10
dup_rank = 9
if dup_rank <= k:
  print((1*1/np.log2(1+dup_rank)).round(1))
else:  
  print((0*1/np.log2(1+dup_rank)).round(1))

0.3


ОТВЕТ 3: DCG@10, если  rank_q′i=9 равен 0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [12]:
def hits_count(dup_ranks, k):
    sum = 0
    for i in dup_ranks: 
      if i <= k:
        sum +=1
    hits_value = sum  / len(dup_ranks) 
    return hits_value    

In [13]:
def dcg_score(dup_ranks, k):
  sum = 0
  for i in dup_ranks: 
      if i <= k:
        sum += 1*1/np.log2(1+i)
      else:  
        sum += 0*1/np.log2(1+i)
  dcg_value = sum/len(dup_ranks)   
  return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [14]:
import pandas as pd

In [15]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [16]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


Отлично, вроде все работает.

### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [17]:
!unzip stackoverflow_similar_questions.zip

unzip:  cannot find or open stackoverflow_similar_questions.zip, stackoverflow_similar_questions.zip.zip or stackoverflow_similar_questions.zip.ZIP.


Считайте данные.

In [47]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
         data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [48]:
validation_data = read_corpus('/content/validation.tsv')

Кол-во строк

In [49]:
len(validation_data)

3760

Размер нескольких первых строк

In [50]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [51]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [52]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):

  vec_que = question_to_vec(question, embeddings, tokenizer)
  vec_cand = [question_to_vec(cand, embeddings, tokenizer) for cand in candidates]
  cos = cosine_similarity([vec_que],vec_cand)
  cos_sort = sorted(enumerate(*cos), key=lambda x: (x[1]), reverse=True)
  
    
  return [(c[0], candidates[c[0]]) for c in cos_sort]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [53]:
questions = ['converting string to list', 'Sending array via Ajax fails', 'You are my best friend'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results'],
              ['Are you hungry?',
               'Is it true story',
               'You are my favourit girl',
               'You are my best friend']]

In [54]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        #print(ranks)
        print()

Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [55]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(1, 'Getting all list items of an unordered list in PHP'), #скрыт
            (0, 'select2 not displaying search results'), #скрыт
            (2, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

ОТВЕТ: 102


Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [56]:
from tqdm.notebook import tqdm

In [57]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [58]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.879
DCG@1000: 0.444 | Hits@1000: 1.000


Вывод: видим, что модель работает не очень хорошо.

### Эмбеддинги, обученные на корпусе похожих вопросов

Улучшите качество модели.
Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [59]:
train_data = read_corpus('/content/train.tsv')

In [60]:
len(train_data)

1000000

In [61]:
words = []
for line in train_data:
  words.append([' '.join(line)]) 
words[5]

['MongoDB error on find() Retrieve only the queried element in an object array in MongoDB collection\n']

In [62]:
len(words)

1000000

In [63]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords, wordnet
wnl = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [64]:
stopWords = set(stopwords.words('english'))
print(stopWords)

{'each', 'i', 'do', 't', 'hasn', 'can', 'to', "that'll", "shan't", 'into', 'that', "you'd", 'had', 'hadn', 'for', 'again', 'has', 'in', 'further', 'it', 'over', 'll', 'above', 'is', 'a', 'against', 'aren', "don't", "hadn't", 'her', 'weren', 'here', 'haven', 've', 'should', "weren't", 'under', 'whom', 'yours', "isn't", 'out', 'most', 'you', 'of', "wasn't", 'with', 'yourself', 'and', 'through', "won't", 'ain', 'hers', 's', 'very', 'both', "haven't", 'o', 'other', 'y', 'off', 'are', 'theirs', 'being', 'by', 'have', 'any', 'wasn', 'herself', 'couldn', 'will', "you'll", 'be', 'while', 'when', 'such', 'because', 'our', "mightn't", 'who', 'these', 'then', 'which', 'am', "needn't", 'only', 'just', 'but', 'between', 'its', "it's", 'myself', 'this', 'so', 'some', 'once', 'ourselves', "hasn't", 'how', 'ours', 'does', 'won', 'needn', 'me', 'doesn', "doesn't", 'about', 'from', "mustn't", 'as', "aren't", 'if', 'himself', 'the', 'we', 'their', 'those', 'him', "you've", 'on', 'they', 'same', 'an', 'th

In [65]:
def preproc_nltk(text):
    #text = re.sub(f'[{string.punctuation}]', ' ', text)
    return ' '.join([wnl.lemmatize(word) for word in word_tokenize(text) if word not in stopWords])


In [66]:
proc_words = [preproc_nltk(t).split() for text in words for t in text]

Обучим модель Word2Vec из gensim. Размер window выберем 15, как написано в руководстве =))

In [67]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(proc_words,  # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=30,             # consider words that occured at least 5 times
                 window=8).wv

In [68]:
len(embeddings_trained.index2word)

18252

In [69]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [70]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.324 | Hits@   1: 0.324
DCG@   5: 0.405 | Hits@   5: 0.479
DCG@  10: 0.434 | Hits@  10: 0.568
DCG@ 100: 0.481 | Hits@ 100: 0.800
DCG@ 500: 0.501 | Hits@ 500: 0.956
DCG@1000: 0.506 | Hits@1000: 1.000


Видим, что результаты улучшились, но не сильно. 

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1. Конечно, nltk.tokenize работает лучше, чем самописный 
2. Нормализация помогает! Особенно удаление знаков пунктуации и других ненужных символов. 
3. Лучше брать скользящее окно 10-15 и частоту вхождения слов около 20-30. 
4. Плохое качество из-за недостаточной обработки (нормализации) входных данных.
5. Предлагается: дополнительно убрать знаки пунктуации, еще поиграться с параметрами Word2Vec, а также попробовать использовать другой лемматизатор или стемминг.

Уберем лишние знаки


In [71]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [72]:
def preproc_nltk_2(text):
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    return ' '.join([wnl.lemmatize(word) for word in word_tokenize(text) if word not in stopWords])


In [73]:
proc_words_2 = [preproc_nltk_2(t).strip().split() for text in words for t in text]

In [74]:
embeddings_trained_2 = Word2Vec(proc_words_2,  # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=30,             # consider words that occured at least 5 times
                 window=10).wv

In [75]:
len(embeddings_trained_2.index2word)

17307

In [76]:
wv_ranking_2 = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks_2 = rank_candidates(q, ex, embeddings_trained_2, tokenizer)
    wv_ranking_2.append([r[0] for r in ranks_2].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [77]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking_2, k), k, hits_count(wv_ranking_2, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.324 | Hits@   1: 0.324
DCG@   5: 0.416 | Hits@   5: 0.498
DCG@  10: 0.442 | Hits@  10: 0.577
DCG@ 100: 0.490 | Hits@ 100: 0.813
DCG@ 500: 0.508 | Hits@ 500: 0.953
DCG@1000: 0.513 | Hits@1000: 1.000


Так себе результаты. Метрики увеличились, но не очень.

Поробуем SnowballStemmer и PorterStemmer

SnowballStemmer вместо nltk.WordNetLemmatizer

In [78]:
from nltk.stem import PorterStemmer, SnowballStemmer
ss = SnowballStemmer(language='english')


In [79]:
def preproc_nltk_3(text):
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    return ' '.join([ss.stem(word) for word in word_tokenize(text) if word not in stopWords])

In [80]:
proc_words_3 = [preproc_nltk_3(t).strip().split() for text in words for t in text]

In [81]:
embeddings_trained_3 = Word2Vec(proc_words_3,  # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=30,             # consider words that occured at least 5 times
                 window=10).wv

In [82]:
len(embeddings_trained_3.index2word)

10022

In [83]:
wv_ranking_3 = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks_3 = rank_candidates(q, ex, embeddings_trained_3, tokenizer)
    wv_ranking_3.append([r[0] for r in ranks_3].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [84]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking_3, k), k, hits_count(wv_ranking_3, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.155 | Hits@   1: 0.155
DCG@   5: 0.196 | Hits@   5: 0.233
DCG@  10: 0.207 | Hits@  10: 0.269
DCG@ 100: 0.247 | Hits@ 100: 0.466
DCG@ 500: 0.284 | Hits@ 500: 0.764
DCG@1000: 0.309 | Hits@1000: 1.000


Вообще печалька =((((


PorterStemmer вместо nltk.WordNetLemmatizer

In [85]:
ps = PorterStemmer()

In [86]:
def preproc_nltk_3(text):
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    return ' '.join([ps.stem(word) for word in word_tokenize(text) if word not in stopWords])

In [87]:
proc_words_3 = [preproc_nltk_3(t).strip().split() for text in words for t in text]

In [88]:
words[:3]

[['converting string to list Convert Google results object (pure js) to Python object\n'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool? Event handling for geometries in Three.js?\n'],
 ['Sending array via Ajax fails Getting all list items of an unordered list in PHP\n']]

In [89]:
proc_words_3[:3]

[['convert',
  'string',
  'list',
  'convert',
  'googl',
  'result',
  'object',
  'pure',
  'js',
  'python',
  'object'],
 ['which',
  'html',
  '5',
  'canva',
  'javascript',
  'use',
  'make',
  'interact',
  'draw',
  'tool',
  'event',
  'handl',
  'geometri',
  'three',
  'js'],
 ['send',
  'array',
  'via',
  'ajax',
  'fail',
  'get',
  'list',
  'item',
  'unord',
  'list',
  'php']]

In [90]:
embeddings_trained_3 = Word2Vec(proc_words_3,  # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=30,             # consider words that occured at least 5 times
                 window=10).wv

In [91]:
len(embeddings_trained_3.index2word)

10045

In [92]:
wv_ranking_3 = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks_3 = rank_candidates(q, ex, embeddings_trained_3, tokenizer)
    wv_ranking_3.append([r[0] for r in ranks_3].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [93]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking_3, k), k, hits_count(wv_ranking_3, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.153 | Hits@   1: 0.153
DCG@   5: 0.193 | Hits@   5: 0.230
DCG@  10: 0.205 | Hits@  10: 0.267
DCG@ 100: 0.244 | Hits@ 100: 0.465
DCG@ 500: 0.283 | Hits@ 500: 0.770
DCG@1000: 0.307 | Hits@1000: 1.000


С PorterStemmer тоже все плохо. Очень. 


Последняя попытка: попробуем WordNetLemmatizer() с POS

In [94]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


In [95]:
def preproc_nltk_4(text):
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    return ' '.join([wnl.lemmatize(word, get_wordnet_pos(word)) for word in word_tokenize(text.lower()) if word not in stopWords])

In [96]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [97]:
proc_words_4 = [preproc_nltk_4(t).strip().split() for text in words for t in text]

In [98]:
words[:3]

[['converting string to list Convert Google results object (pure js) to Python object\n'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool? Event handling for geometries in Three.js?\n'],
 ['Sending array via Ajax fails Getting all list items of an unordered list in PHP\n']]

In [99]:
proc_words_4[:3]

[['convert',
  'string',
  'list',
  'convert',
  'google',
  'result',
  'object',
  'pure',
  'j',
  'python',
  'object'],
 ['html',
  '5',
  'canvas',
  'javascript',
  'use',
  'make',
  'interactive',
  'draw',
  'tool',
  'event',
  'handle',
  'geometry',
  'three',
  'j'],
 ['send',
  'array',
  'via',
  'ajax',
  'fails',
  'get',
  'list',
  'item',
  'unordered',
  'list',
  'php']]

In [100]:
embeddings_trained_4 = Word2Vec(proc_words_4,  # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=30,             # consider words that occured at least 5 times
                 window=10).wv

In [101]:
len(embeddings_trained_4.index2word)

11067

In [102]:
wv_ranking_4 = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks_4 = rank_candidates(q, ex, embeddings_trained_4, tokenizer)
    wv_ranking_4.append([r[0] for r in ranks_4].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [103]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking_4, k), k, hits_count(wv_ranking_4, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.267 | Hits@   1: 0.267
DCG@   5: 0.326 | Hits@   5: 0.380
DCG@  10: 0.346 | Hits@  10: 0.440
DCG@ 100: 0.387 | Hits@ 100: 0.647
DCG@ 500: 0.413 | Hits@ 500: 0.845
DCG@1000: 0.429 | Hits@1000: 1.000


Вывод: 
В общем, хорошо задачу решить не удалось. 

Максимальные полученные информативные метрики получились при использовании лемматизации без POS, при удаленных знаках препинания, min_count=10, window=15: 

DCG@1000 = 0.521.

Hits@1 = 0.338. 

Похоже, надо больше экспериментов!